Tasks done (in order):
- Get drug reaction info (only look at top 5 reactions with most data)   
- Get protein cell level info   
- Get the IC50 information from TDC and ConPLex
- Combine the dataframes using multiplication  

These tasks are done in a separate file:
- Get MACCS fingerprint  
- Create the train test split and dataloaders  
- Create the models  
- Train the model  
- Make some predictions  

# Data collection

## Get the drug info

### Get our reaction data

In [1]:
import pandas as pd

reaction_df = pd.read_table('drug_reaction_freq.tsv')
reaction_df = reaction_df[['drug', 'diarrhoea', 'headache', 'nausea', 'vomiting', 'dizziness']]
reaction_df

,drug,diarrhoea,headache,nausea,vomiting,dizziness
0,alfentanil,-100,-100,5,5,4
1,telithromycin,4,4,4,4,4
2,simeprevir,-100,5,5,-100,-100
3,pentamidine.isethionate,-100,-100,-100,-100,5
4,penicillamine,-100,-100,-100,-100,-100
...,...,...,...,...,...,...
754,rufinamide,-100,5,4,4,4
755,tafluprost,-100,4,-100,-100,-100
756,cabazitaxel,4,1,4,4,3
757,aclidinium.bromide,4,4,-100,4,-100


In [2]:
start_col = reaction_df.columns.get_loc('diarrhoea')
end_col = reaction_df.columns.get_loc('dizziness')
for i in range(start_col, end_col+1):
    col = reaction_df.columns[i]
    a = list(reaction_df[col] != -100)
    print(a.count(True))

473
516
558
455
490


### Get the Pubchem IDs (needed for IC50)

In [3]:
cid_df = pd.read_table('drug_cid.tsv')
drug_to_cid = dict()
for ind in cid_df.index:
    drug_to_cid[cid_df['GenericName'][ind]] = cid_df['CID'][ind]
print(len(drug_to_cid.keys()))
cids = set(drug_to_cid.values())
print(cids)

759
{208898, 208902, 6918155, 16220172, 208908, 25151504, 57363, 6167, 44146714, 32797, 2082, 55331, 2083, 2088, 110634, 110635, 2092, 5328940, 2094, 213039, 2099, 9852981, 213046, 16136245, 9871419, 6918204, 84029, 4158, 51263, 11626560, 2118, 4170, 4171, 9578572, 4173, 446541, 4174, 4178, 2130, 4184, 125017, 5353562, 65628, 92253, 6238, 4189, 4192, 4195, 2153, 6918250, 6251, 6252, 4205, 2156, 2159, 2161, 2162, 4212, 57469, 39042, 2179, 446596, 26757, 2182, 9926791, 137, 2187, 4236, 6918289, 16130199, 26275995, 5273759, 77990, 2216, 16132265, 77993, 77992, 6918313, 10413, 77998, 4274, 5484727, 55480, 129211, 2249, 5329098, 129228, 5329102, 47319, 2265, 2266, 2267, 157920, 157921, 82146, 82148, 6850789, 5288169, 25077993, 39147, 82153, 2284, 5742832, 358641, 247, 12536, 5361917, 3010818, 39186, 6432, 10531, 104741, 6435110, 448812, 2351, 65840, 160051, 6918453, 104758, 9865528, 10182969, 4409, 6918456, 2366, 65856, 45375808, 16132418, 2369, 5462337, 2375, 65866, 3086674, 4436, 153941, 

## Get the protein info

### Get tissue expression level

In [24]:
import json

with open("prot_cell_levels.json") as f:
    cell_level = json.load(f)
prots = list(cell_level.keys())
print(len(prots))

15004


### Get the IC50 values

In [5]:
ic_df1 = pd.read_csv('train_val.csv')
ic_df2 = pd.read_csv('test.csv')
ic_df = pd.concat([ic_df1, ic_df2], ignore_index=True)
ic_df = ic_df.drop(columns=['Year'])
ic_df = ic_df[['Drug_ID', 'Target_ID', 'Target', 'Y']]
print(f"unique proteins (in Uniprot ID): {len(ic_df['Target_ID'].unique())}")
print(f"unique drugs (in Pubchem ID): {len(ic_df['Drug_ID'].unique())}")
uniprot_ids = set(ic_df['Target_ID'].unique())

ic_df_subset = ic_df[ic_df['Drug_ID'].isin(cids)]
print(f"unique drugs that also have side effect info: {len(ic_df_subset['Drug_ID'].unique())}") ## This value is too small, use the full dataset from TDC
ic_df_subset

unique proteins (in Uniprot ID): 477
unique drugs (in Pubchem ID): 139739
unique drugs that also have side effect info: 64


,Drug_ID,Target_ID,Target,Y
12784,3365.0,P11712,MDSLVVLVLCLSCLLLLSLWRQSSGRGKLPPGPTPLPVIGNILQIG...,10.275051
12785,71616.0,P11712,MDSLVVLVLCLSCLLLLSLWRQSSGRGKLPPGPTPLPVIGNILQIG...,9.546813
12788,3365.0,P33261,MDPFVVLVLCLSCLLLLSIWRQSSGRGKLPPGPTPLPVIGNILQID...,9.011889
12789,71616.0,P33261,MDPFVVLVLCLSCLLLLSIWRQSSGRGKLPPGPTPLPVIGNILQID...,9.615805
12792,3365.0,P08684,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,8.987197
...,...,...,...,...
232092,42611257.0,P15056,MAALSGGGGGGAEPGQALFNGDMEPEAGAGAGAAASSAADPAIPEE...,4.605170
232165,11167602.0,P07949,MAKATSGAAGLRLLLLLLLPLLGKVALGLYFSRDAYWEKLYVDQAA...,2.484907
232166,216239.0,P07949,MAKATSGAAGLRLLLLLLLPLLGKVALGLYFSRDAYWEKLYVDQAA...,2.066863
232167,3081361.0,P07949,MAKATSGAAGLRLLLLLLLPLLGKVALGLYFSRDAYWEKLYVDQAA...,1.386294


In [6]:
from tdc.multi_pred import DTI 

data = DTI(name = "BindingDB_IC50")
data.harmonize_affinities(mode = "mean")
data.convert_to_log(form = "binding")
split = data.get_split()
full_df = pd.concat([split['train'], split['test'], split['valid']], ignore_index=True)
full_df = full_df[full_df['Drug_ID'].isin(cids)]
full_df = full_df[full_df['Target_ID'].isin(uniprot_ids)]
full_df = full_df.drop(columns=['Target'])
print(f"unique drugs that also have side effect data: {len(full_df['Drug_ID'].unique())}") # Value is also quite low
full_df

Found local copy...
Loading...
Done!
The original data has been updated!
To log space...


unique drugs that also have side effect data: 281


,Drug_ID,Drug,Target_ID,Y
133,444.0,CC(NC(C)(C)C)C(=O)c1cccc(Cl)c1,P23975,5.843542
135,444.0,CC(NC(C)(C)C)C(=O)c1cccc(Cl)c1,P31645,4.434443
137,444.0,CC(NC(C)(C)C)C(=O)c1cccc(Cl)c1,Q01959,5.948203
433,1727.0,Nc1ccncc1,Q12809,2.358270
444,1775.0,O=C1N=C(O)NC1(c1ccccc1)c1ccccc1,Q13936,3.903090
...,...,...,...,...
765018,3108.0,OCCN(CCO)c1nc(N2CCCCC2)c2nc(N(CCO)CCO)nc(N3CCC...,O15244,5.585010
765948,5360515.0,O=C1CC[C@@]2(O)[C@H]3Cc4ccc(O)c5c4[C@@]2(CCN3C...,P35372,8.062231
766024,16362.0,O=c1[nH]c2ccccc2n1C1CCN(CCCC(c2ccc(F)cc2)c2ccc...,P35372,6.429340
766113,3826.0,O=C(c1ccccc1)c1ccc2n1CCC2C(=O)O,P42574,5.602043


In [7]:
affinities = dict()

known_dti_pairs = set()
for ind in full_df.index:
    drug_cid = full_df['Drug_ID'][ind]
    prot_uid = full_df['Target_ID'][ind]
    known_dti_pairs.add((drug_cid, prot_uid))
    affinity = full_df['Y'][ind]
    affinities[(drug_cid, prot_uid)] = affinity

FileNotFoundError: [Errno 2] No such file or directory: 'cid_smiles_final.txt'

### Using ConPLex to predict IC50 values

In [ ]:
# create query df (this doesn't need to run after the first time)
cid_to_smiles = {}
with open("cid_smiles_final.txt") as f:
    for line in f.readlines():
        words = line.split()
        cid = int(words[0])
        cid_to_smiles[cid] = words[1]
uid_to_sequence = {}
for ind in ic_df.index:
    uid = ic_df['Target_ID'][ind]
    seq = ic_df['Target'][ind]
    uid_to_sequence[uid] = seq
print(f"drugs: {len(cid_to_smiles.keys())}")
print(f"prots: {len(uid_to_sequence.keys())}")

query_df = {"proteinID": list(),
            "moleculeID": list(),
            "proteinSequence": list(),
            "moleculeSmiles": list()
}
for cid in cids:
    for uid in uniprot_ids:
        if (cid, uid) in known_dti_pairs:
            continue 
        smiles = cid_to_smiles[cid]
        sequence = uid_to_sequence[uid]
        query_df['proteinID'].append(uid)
        query_df['moleculeID'].append(cid)
        query_df['proteinSequence'].append(sequence)
        query_df['moleculeSmiles'].append(smiles)
query_df = pd.DataFrame.from_dict(query_df)
# query_df.to_csv('query_df.tsv', sep='\t', index=False)

In [10]:
# Use ConPLex to predict the other pIC50 vals
from tqdm import tqdm

from conplex_dti.model.architectures import SimpleCoembeddingNoSigmoid
from conplex_dti.featurizer.protein import ProtBertFeaturizer
from conplex_dti.featurizer.molecule import MorganFeaturizer
import torch
from torch.utils.data import DataLoader
import numpy as np

device = torch.device("cpu")
print("Loading models")
target_featurizer = ProtBertFeaturizer(
    save_dir='.', per_tok=False
).cpu()
drug_featurizer = MorganFeaturizer(save_dir='.').cpu()

drug_featurizer.preload(query_df["moleculeSmiles"].unique())
target_featurizer.preload(query_df["proteinSequence"].unique())

model = SimpleCoembeddingNoSigmoid(
    drug_featurizer.shape, target_featurizer.shape, 
    latent_dimension=1024,
    latent_activation="GELU",
    latent_distance="Cosine",
    classify=False
)
model_path = './models/Affinity82923_best_model.pt'
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.eval()
model = model.to(device)

dt_feature_pairs = [
        (drug_featurizer(r["moleculeSmiles"]), target_featurizer(r["proteinSequence"]))
        for _, r in tqdm(query_df.iterrows(), desc="Calculating feature pairs", total=query_df.shape[0])
    ]
dloader = DataLoader(dt_feature_pairs, batch_size=32, shuffle=False)

print(f"Generating predictions...")
preds = []
with torch.set_grad_enabled(False):
    for b in tqdm(dloader, desc="Calculating predictions"):
        preds.append(model(b[0], b[1]).detach().cpu().numpy())

preds = np.concatenate(preds)

result_df = pd.DataFrame(query_df[["moleculeID", "proteinID"]])
result_df["Prediction"] = preds

Loading models


Calculating feature pairs: 100%|██████████| 361071/361071 [00:04<00:00, 73220.78it/s]


Generating predictions...


Calculating predictions: 100%|██████████| 11284/11284 [00:19<00:00, 581.66it/s]


In [11]:
result_df.to_csv('query_df_results.tsv', index=False)

### Read IC50 values

In [10]:
result_df = pd.read_csv('query_df_results.csv')
for ind in result_df.index:
    cid = result_df['moleculeID'][ind]
    uid = result_df['proteinID'][ind]
    affinity = result_df['Prediction'][ind]
    affinities[(cid, uid)] = affinity

In [15]:
import json

drug_prot_ic = {}
with open("uid_to_prot.json") as f:
    uniprot_to_prot = json.load(f)
cid_to_drug = {}
for drug in drug_to_cid.keys():
    cid_to_drug[drug_to_cid[drug]] = drug

for cid in cids:
    drug = cid_to_drug[cid]
    drug_prot_ic[drug] = {}
    for uid in uniprot_ids:
        affinity = affinities[(cid, uid)]
        prot = uniprot_to_prot[uid]
        drug_prot_ic[drug][prot] = affinity
# Just to make sure
print(f"total distinct proteins: {len(drug_prot_ic['alfentanil'].keys())}")
print(f"total distinct drugs: {len(drug_prot_ic.keys())}")

with open("drug_prot_ic.json", "w") as f:
    json.dump(drug_prot_ic, f, indent=4)

total distinct proteins: 477
total distinct drugs: 759


In [26]:
# Export to a dataframe
drugs = set(drug_prot_ic.keys())
prots = set(drug_prot_ic['alfentanil'].keys())

drug_ic_df = pd.read_csv('drug_smiles.csv')
for prot in prots:
    drug_ic_df[prot] = drug_ic_df['drug'].apply(lambda x : drug_prot_ic[x][prot])
drug_ic_df.to_csv('drug_smiles_ic50.csv', index=False)

# Data cleaning

Dataframes that are the inputs:
- reaction_df
- drug_ic_df
- cell_level

Miscellaneous stuff that should be correct:
- drugs
- drug_prot_ic
- prots

## Get cell 'strength' and prepare data for ML

In [30]:
from tqdm import tqdm

cells = set(cell_level['PLCL2'].keys())
cells.remove("nan")
print(len(cells))
drug_strength = {}
drug_strength['drug'] = list()
drug_strength['SMILES'] = list()
for cell in cells:
    drug_strength[cell] = list()

# Get the proteins that have cell level info
correct_prots = set()
for prot in prots:
    if prot in cell_level.keys():
        correct_prots.add(prot)
print(f"proteins that have expression level and ic info: {len(correct_prots)}")
prots = correct_prots

for ind in tqdm(drug_ic_df.index, total=drug_ic_df.shape[0], desc='Calculate cell strength'):
    drug = drug_ic_df['drug'][ind]
    drug_strength['drug'].append(drug)
    smiles = drug_ic_df['SMILES'][ind]
    drug_strength['SMILES'].append(smiles)
    for cell in cells:
        total = -100
        for prot in prots:
            ic = drug_ic_df[prot][ind]
            level = cell_level[prot][cell]
            ans = -100
            try:
                if level >= 0:
                    ans = level * ic
            except:
                print(f"level not found for {prot} and cell {cell}")
            if ans != -100:
                if total == -100:
                    total = ans
                else:
                    total += ans
        drug_strength[cell].append(total)
drug_strength = pd.DataFrame.from_dict(drug_strength) 
drug_strength


260
proteins that have expression level and ic info: 410


Calculate cell strength: 100%|██████████| 759/759 [02:55<00:00,  4.34it/s]


,drug,SMILES,appendix endocrine cells,dorsal raphe glial cells,urinary bladder urothelial cells,colon mucosal lymphoid cells,skeletal muscle myocytes,nasopharynx ciliated cells (ciliary rootlets),retina pigment epithelial cells,skin 2 extracellular matrix,...,lung alveolar cells type i,cerebellum synaptic glomeruli - capsule,liver cholangiocytes,bronchus goblet cells,cerebellum processes in white matter,endometrium nonciliated luminal epithelial cells,skin 1 arrector pili muscle cells,endometrium macrophages,retina inner plexiform layer,appendix germinal center cells
0,alfentanil,CCC(=O)N(C1=CC=CC=C1)C2(CCN(CC2)CCN3C(=O)N(N=N...,222.636474,0.0,2081.465124,189.414262,1223.902947,77.474186,0.0,4.960150,...,-100,18.663531,898.330240,36.020592,58.475536,16.275451,16.637873,6.168876,0.0,118.175929
1,telithromycin,CC[C@@H]1[C@@]2([C@@H]([C@H](C(=O)[C@@H](C[C@@...,195.324457,0.0,1831.329868,181.184358,1033.228964,69.166455,0.0,6.614801,...,-100,33.199048,776.815161,30.107373,68.693203,15.736785,11.285045,8.357745,0.0,103.563673
2,simeprevir,CC1=C(C=CC2=C1N=C(C=C2O[C@@H]3C[C@@H]4[C@@H](C...,178.337076,0.0,1673.227479,179.020611,941.742490,69.372739,0.0,4.244746,...,-100,21.405974,707.386513,22.843688,53.993074,19.902229,12.183476,12.193711,0.0,89.747383
3,pentamidine.isethionate,C1=CC(=CC=C1C(=N)N)OCCCCCOC2=CC=C(C=C2)C(=N)N,340.393858,0.0,3335.973494,332.928487,1889.015122,97.923374,0.0,11.970572,...,-100,52.061981,1435.705153,75.637824,99.621766,21.021722,26.810649,11.170288,0.0,180.489427
4,penicillamine,CC(C)([C@H](C(=O)O)N)S,326.400014,0.0,3169.440428,302.006056,1801.714526,88.047181,0.0,10.791633,...,-100,55.876451,1363.935845,80.235924,100.973039,20.390096,28.143227,10.600160,0.0,169.467455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,rufinamide,C1=CC(=C(C(=C1)F)CN2C=C(N=N2)C(=O)N)F,345.034223,0.0,3218.023085,310.913601,1815.819738,85.490402,0.0,10.908236,...,-100,55.816379,1385.553969,72.974011,104.042586,19.726612,30.311822,9.632385,0.0,179.731821
755,tafluprost,CC(C)OC(=O)CCC/C=C\C[C@H]1[C@H](C[C@H]([C@@H]1...,398.349503,0.0,3522.144581,384.124488,1996.535424,120.416548,0.0,13.672210,...,-100,48.870875,1551.927731,76.271217,105.715540,25.575556,34.390862,15.707435,0.0,196.448277
756,cabazitaxel,CC1=C2[C@H](C(=O)[C@@]3([C@H](C[C@@H]4[C@]([C@...,135.970640,0.0,1428.035175,129.718798,848.057862,63.097601,0.0,6.917483,...,-100,12.700830,617.851146,28.650367,39.373380,8.471188,10.596793,2.410871,0.0,85.159227
757,aclidinium.bromide,C1C[N+]2(CCC1C(C2)OC(=O)C(C3=CC=CS3)(C4=CC=CS4...,219.669607,0.0,2083.348403,203.427404,1214.863068,82.129505,0.0,9.863575,...,-100,21.095221,917.905406,39.587632,51.493471,7.696042,19.558767,2.548494,0.0,118.436699


In [32]:
full_df = drug_strength.merge(reaction_df)
full_df.to_csv('drug_ml_info.csv', index=False)